# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [87]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

# data_dir = './data/simpsons/moes_tavern_lines.txt'
data_dir = 'data/simpsons.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [88]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [89]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    # Split all the text into words - All the possible words collections/choices
    #words = text.split() # This is not the unique words
    #words = {word: None for word in text.split()}
    #words = token_lookup()
    
    from collections import Counter
    # Count the freq of words in the text/collection of words
    word_counts = Counter(text)
    
    # Having counted the frequency of the words in collection, sort them from most to least/top to bottom/descendng
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    
    # first enumerating for vocab to int
    vocab_to_int = {words: ii for ii, words in enumerate(sorted_vocab)}
    
    # into_to_vocab after enumerating through the sorted vocab
    int_to_vocab = {ii: words for words, ii in vocab_to_int.items()}
    
    # return the output results: a tuple of dicts(vocab_to_int, int_to_vocab)
    # return dicts(vocab_to_int, int_to_vocab)
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [90]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
   
    # Replace punctuation with tokens so we can use them in our model
    token_dict = {'.': '||Period||',
                  ',': '||Comma||', 
                  '"': '||Quotation_Mark||', 
                  ';': '||Semicolon||', 
                  '!': '||Exclamation_Mark||', 
                  '?': '||Question_Mark||', 
                  '(': '||Left_Parentheses||', 
                  ')': '||Right_Parentheses||', 
                  '--': '||Dash||', 
                  '\n': '||Return||'}
    #token_dict.items() # to show it
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [91]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [92]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [93]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


/home/arasdar/anaconda3/envs/arasdar-DL-env/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [94]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input')
    targets = tf.placeholder(dtype=tf.int32, shape=[None, None], name='targets')
    lr = tf.placeholder(dtype=tf.float32, shape=None, name='learning_rate')
    return input, targets, lr


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


# Writing a custom RNN cell
It’s almost too easy to use the standard GRU or LSTM cells, so let’s define our own RNN cell. 
Here’s a random idea that may or may not work: starting with a GRU cell, instead of taking a single transformation of its input, we enable it to take a weighted average of multiple transformations of its input. 
The idea is that we might benefit from treat the input differently in different scenarios (e.g., we may want to treat verbs differently than nouns).

To write the custom cell, we need to extend tf.nn.rnn_cell.RNNCell. 
Specifically, we need to fill in 3 abstract methods and write an __init__ method. 
First, let’s start with a GRU cell, adapted from Tensorflow’s implementation:

In [123]:
# class GRUCell(tf.nn.rnn_cell.RNNCell): # this probablly belongs to the old version
class GRUCell(tf.contrib.rnn.RNNCell):
    """Gated Recurrent Unit cell (cf. http://arxiv.org/abs/1406.1078)."""

    def __init__(self, num_units):
        self._num_units = num_units

    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        with tf.variable_scope(scope or type(self).__name__):  # "GRUCell"
            with tf.variable_scope("Gates"):  # Reset gate and update gate.
                # We start with bias of 1.0 to not reset and not update.
#                 ru = tf.nn.rnn_cell._linear([inputs, state],
#                                         2 * self._num_units, True, 1.0)
#                 ru = tf.contrib.layers.linear([inputs, state], 2 * self._num_units, True, 1.0)
                ru = tf.contrib.layers.fully_connected(inputs=[inputs, state], 
                                                       num_outputs=(2 * self._num_units), 
                                                       activation_fn=None # Linear Units (LU)
                                                      )
                ru = tf.nn.sigmoid(ru)
                r, u = tf.split(axis=1, num_or_size_splits=2, value=ru, name='split', num=None) # opposite of concatenation
            with tf.variable_scope("Candidate"):
#                 c = tf.nn.tanh(tf.nn.rnn_cell._linear([inputs, r * state],
#                                              self._num_units, True))
#                 tf.contrib.layers.linear([inputs, r * state], self._num_units, True)
                logits = tf.contrib.layers.fully_connected(inputs=[inputs, r * state], 
                                                           num_outputs=self._num_units, 
                                                           activation_fn=None)
                c = tf.nn.tanh(x=logits)
            new_h = u * state + (1 - u) * c
        return new_h, new_h

# CustomRNNCell
We modify the __init__ method to take a parameter at initialization, which will determine the number of transformation matrices.
def __init__(self, num_units, num_weights):
    self._num_units = num_units
    self._num_weights = num_weights
Then, we modify the Candidate variable scope of the __call__ method to do a weighted average as shown below (note that all of the matrices are created as a single variable and then split into multiple tensors):

In [124]:
# class CustomCell(tf.nn.rnn_cell.RNNCell):
class CustomRNNCell(tf.contrib.rnn.RNNCell):
    """Gated Recurrent Unit cell (cf. http://arxiv.org/abs/1406.1078)."""

    def __init__(self, num_units, num_weights):
        self._num_units = num_units
        self._num_weights = num_weights

    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        with tf.variable_scope(scope or type(self).__name__):  # "GRUCell"
            with tf.variable_scope("Gates"):  # Reset gate and update gate.
                # We start with bias of 1.0 to not reset and not update.
#                 ru = tf.nn.rnn_cell._linear([inputs, state],
#                                         2 * self._num_units, True, 1.0)
                ru = tf.contrib.layers.linear([inputs, state], 2 * self._num_units, True, 1.0)

                ru = tf.nn.sigmoid(ru)
                r, u = tf.split(1, 2, ru)
            with tf.variable_scope("Candidate"):
#                 lambdas = tf.nn.rnn_cell._linear([inputs, state], self._num_weights, True)
                # tf.contrib.layers.linear
                lambdas = tf.contrib.layers.linear([inputs, state], self._num_weights, True)

                lambdas = tf.split(1, self._num_weights, tf.nn.softmax(lambdas))

                Ws = tf.get_variable("Ws", shape = [self._num_weights, inputs.get_shape()[1], self._num_units])
                Ws = [tf.squeeze(i) for i in tf.split(0, self._num_weights, Ws)]

                candidate_inputs = []

                for idx, W in enumerate(Ws):
                    candidate_inputs.append(tf.matmul(inputs, W) * lambdas[idx])

                Wx = tf.add_n(candidate_inputs)
                
#                 c = tf.nn.tanh(Wx + tf.nn.rnn_cell._linear([r * state],
#                                             self._num_units, True, scope="second"))
                b = tf.contrib.layers.linear([r * state], self._num_units, True, scope="second")
                c = tf.nn.tanh(Wx+b)
            new_h = u * state + (1 - u) * c
        return new_h, new_h

### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [132]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    # The number of LSTM cells/ Memory cells in one layer for RNN
#     rnn = tf.contrib.rnn.BasicLSTMCell(rnn_size) # rnn_size==LSTM_size?? 
#     rnn = tf.contrib.rnn.GRUCell(num_units=rnn_size)
#     rnn = GRUCell(num_units=rnn_size)
    rnn = tf.contrib.rnn.BasicRNNCell(num_units=rnn_size)
    
    #     # Adding Dropout NOT needed/ Not Asked
    #     keep_prob = 1.0 # Drop out probability
    #     drop = tf.contrib.rnn.DropoutWrapper(rnn, keep_prob) #output_keep_prop=
    
    # Stacking up multiple LSTM layers for DL
    rnn_layers = 10 # layers
    cell = tf.contrib.rnn.MultiRNNCell([rnn] * rnn_layers)
    
    # Initializing the cell state using zero_state()
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(input=initial_state, name='initial_state')

    return cell, initial_state

# Aras: Already implemented in sentiment network
# lstm_size = 256
# lstm_layers = 1
# batch_size = 500
# learning_rate = 0.001

# with graph.as_default():
#     # Your basic LSTM cell
#     lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
#     # Add dropout to the cell
#     drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
#     # Stack up multiple LSTM layers, for deep learning
#     cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
#     # Getting an initial state of all zerosoutput_keep_prop
#     initial_state = cell.zero_state(batch_size, tf.float32)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [133]:
def get_embed(input_data, vocab_size, embed_dim):
    """initial_state
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    # Size of embedding vectors (number of units in the emdding layer)
    # embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embedding = tf.Variable(tf.random_uniform(shape=[vocab_size, embed_dim], minval=-1, maxval=1, dtype=tf.float32, 
                                  seed=None, name=None))
    #     tf.random_normal(mean=1.0, size/shape=[], stddev=0.1)
    # tf.random_normal(shape=[vocab_size/n_words, embed_size/embed_dim], mean=0.0, stddev=1.0, 
                     #dtype=tf.float32, seed=None, name=None)
        
    embed = tf.nn.embedding_lookup(embedding, input_data)

    return embed

# # Embedding implementation from Sentiment_RNN_solution.ipynb
# # Size of the embedding vectors (number of units in the embedding layer)
# embed_size = 300 

# with graph.as_default():
#     embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
#     embed = tf.nn.embedding_lookup(embedding, inputs_)tf.nn.dynamic_rnn

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [134]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    # Create the RNN using the cells and the embedded input vectors
    # outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=cell.state)
    # initial_state=["initial_state"]
    outputs, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs, 
                                             sequence_length=None, 
                                             initial_state=None, 
                                             dtype=tf.float32, parallel_iterations=None, 
                                             swap_memory=False, time_major=False, scope=None)
    
    # Naming the final_state using tf.identity(input, name)
    final_state = tf.identity(input=final_state, name='final_state')
    
    # Returning the outputs and the final_state
    return outputs, final_state

# Aras: Implementation from Sentiment_RNN_Solution.ipynb
# with graph.as_default():
#     outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
#                                              initial_state=initial_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [135]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    # embedding layer: def get_embed(input_data, vocab_size, embed_dim):
    embed = get_embed(input_data=input_data, vocab_size=vocab_size, embed_dim=rnn_size)
    
    # build rnn: def build_rnn(cell, inputs):
    outputs, final_state = build_rnn(cell=cell, inputs=embed)
    
    # Apply a fully connected layer with a linear activation and vocab_size as the number of outputs.
    logits = tf.contrib.layers.fully_connected(inputs=outputs, num_outputs=vocab_size, activation_fn=None)
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [136]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    # calculating the batch length, i.e. number words in a batch
    batch_length = batch_size*seq_length # Remeber: batch_length != batch_size
    
    # Number of bacthes in the give text with word IDs
    num_batches = len(int_text)// batch_length
    if (len(int_text)//batch_length) == (len(int_text)/batch_length):
        num_batches -= 1
    
    # preparing the numpy array first which is going to be returned/outputed
    batches = np.zeros([num_batches, 2, batch_size, seq_length])
    
    # number of words in the text (out dataset)
    # get rid of the rest of the text which can fully be included in a batch based on the batch size
    int_text = int_text[:(num_batches*batch_length)+1] # incremented one for the IO sequences/seq2seq learning

    # Now based on the txt_size, batch_size, and seq_size/length, we should start getting the batches stochastically
    #for batch_index/b_idx in range(start=0, stop=len(int_text), step=batch_size):
    for batch_idx in range(0, num_batches, 1):
        batch_slice = int_text[batch_idx*batch_length:(batch_idx+1)*batch_length+1]
    
        # Slicing up the sequences inside a batch
        #for seq_index/s_idx in range(start=0, stop=len(batch[??]), step=seq_length): # remember each sequence has two seq: input & output
        for seq_idx in range(0, batch_size, 1):
            batches[batch_idx, 0, seq_idx] = batch_slice[seq_idx*seq_length:(seq_idx+1)*seq_length]
            batches[batch_idx, 1, seq_idx] = batch_slice[seq_idx*seq_length+1:((seq_idx+1)*seq_length)+1]
        
    return batches
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [137]:
# Number of Epochs
num_epochs = 100 # depends on how fast the system is and how long we can wait to see the results
# Batch Size
batch_size = 64 # depends on the memory, num seq per batch
# RNN Size
rnn_size = 128 # Pixel and int/8 Bit
# Sequence Length
seq_length = 64 # the same as RNN width size/ number of mem cells in one layer
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 16 # 2^4 show every 16 batches learning/training

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [138]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)
#     capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
#     train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [139]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/16   train_loss = 8.826
Epoch   1 Batch    0/16   train_loss = 6.541
Epoch   2 Batch    0/16   train_loss = 6.194
Epoch   3 Batch    0/16   train_loss = 6.154
Epoch   4 Batch    0/16   train_loss = 6.153
Epoch   5 Batch    0/16   train_loss = 6.193
Epoch   6 Batch    0/16   train_loss = 6.192
Epoch   7 Batch    0/16   train_loss = 6.154
Epoch   8 Batch    0/16   train_loss = 6.262
Epoch   9 Batch    0/16   train_loss = 6.195
Epoch  10 Batch    0/16   train_loss = 6.334
Epoch  11 Batch    0/16   train_loss = 6.352
Epoch  12 Batch    0/16   train_loss = 6.970
Epoch  13 Batch    0/16   train_loss = 6.152
Epoch  14 Batch    0/16   train_loss = 6.375
Epoch  15 Batch    0/16   train_loss = 6.321
Epoch  16 Batch    0/16   train_loss = 6.334
Epoch  17 Batch    0/16   train_loss = 6.195
Epoch  18 Batch    0/16   train_loss = 6.162
Epoch  19 Batch    0/16   train_loss = 6.153
Epoch  20 Batch    0/16   train_loss = 6.149
Epoch  21 Batch    0/16   train_loss = 6.152
Epoch  22 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [82]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [83]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [84]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input = loaded_graph.get_tensor_by_name(name='input:0')
    initial_state = loaded_graph.get_tensor_by_name(name='initial_state:0')
    final_state = loaded_graph.get_tensor_by_name(name='final_state:0')
    probs = loaded_graph.get_tensor_by_name(name='probs:0')
    
    return input, initial_state, final_state, probs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [85]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
    # extractin the string/words out of the int_to_vocab.items
    words = np.array([words for ids, words in int_to_vocab.items()])

    # The generated random samples = numpy.random.choice(a, size=None, replace=True, p=None)¶
    random_word = np.random.choice(a = words, size=None, replace=True, p=probabilities)

    return random_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [86]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:
!. week back like could informant so
my
new.. in i to moe_szyslak: his
moonlight in said
amazing is, excuse my planning booze before." apu_nahasapeemapetilon:. lenny_leonard: me" looting the wanna best)! fight to)? 'cause there's seymour_skinner: it. drunk. you.
! why marge_simpson: don't a you what don't star)? owner pernt. beer. out, time incredible a?? thumb heard
. you, moe_szyslak: lenny_leonard: what from,,
the, comes ya to eye
i've talk but whee sure not better when.. your multi-national a out a can here dead but. impressed
too" to who her give idea, drains
so funny special. the. me and i homer_simpson: is from. folk
more ireland you you've
news don't ground
you putting don't house the nice yeah is., new lenny_leonard: eyeball there i'm being a check important boy
) the i'm-so-stupid hans:. but except not back. y-you carl_carlson: money game in carl_carlson: and! me hope


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.